In [1]:
from llama_index.llms.ollama import Ollama
llm = Ollama(model="llama3", request_timeout=120.0)

In [18]:
import pandas as pd
from tqdm import tqdm
import os
from itertools import combinations

DEV_META = "/home/akhil/models/DCASE24/dcase2024-task6-baseline/data/CLOTHO_v2.1/clotho_csv_files/clotho_captions_development.csv"
EVAL_META = "/home/akhil/models/DCASE24/dcase2024-task6-baseline/data/CLOTHO_v2.1/clotho_csv_files/clotho_captions_evaluation.csv"

In [19]:
direc = '/home/akhil/models/DCASE24/dcase2024-task6-baseline/data/Clotho_newCaps'

In [20]:
PROMPT_TEXT = """Combine the two captions in such a way that their auditory meanings is maintained. Return only one caption without any additional text or information :{},{}"""
get_prompt = lambda sentences: PROMPT_TEXT.format(sentences[0], sentences[1])

In [22]:
def create_new_df(filename, savename):
    # read_csv
    df = pd.read_csv(filename)

    # Make new columns
    for i in range(10):
        df[f'caption_{i+6}']= ''

    print(df.iterrows())
    print(df.shape[0])

    # Generate the new captions
    with tqdm(total=df.shape[0]) as pbar:
        for index, row in enumerate(df.iterrows()):
            captions_list = [row[f'caption-{i}'] for i in range(1,6)]
            counter = 0

            for combo in combinations(captions_list, 2):
                prompt = get_prompt(combo)
                response = str(llm.complete(prompt))
                
                print(f'{combo}\n\n{response}')
                print('-'*30)
                
                df.loc[index, f'caption_{6+counter}'] = response
                counter += 1
                
            pbar.update(1)
    # Save the captions
    df.to_csv(os.path.join(direc, savename))

In [23]:
create_new_df(DEV_META, 'random.csv')

<generator object DataFrame.iterrows at 0x7f014eba3560>
3839


  0%|          | 0/3839 [00:00<?, ?it/s]

TypeError: tuple indices must be integers or slices, not str